In [1]:
from pathlib import Path
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Path to project root
PROJECT_ROOT = Path.home() / "Desktop" / "Insurance_Project"

# Add root to Python path
sys.path.append(str(PROJECT_ROOT))

In [2]:
from pathlib import Path
import os
import importlib
from src.data import preprocessing as pr
import re
file_path = PROJECT_ROOT/"data"/"raw"/"insurance_claims.csv"
data = pr.load_data(file_path)
# Feature engineering specific to the prototype dataset
data = data.drop('_c39', axis=1)
data.replace('?', np.nan, inplace=True)

for char in ['Y', 'YES']:
    data.replace(char, True, inplace=True)
for char in ['N', 'NO']:
    data.replace(char, False, inplace=True)


C:\Users\phili\AppData\Local\Temp\ipykernel_4812\438751034.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(char, False, inplace=True)


In [3]:
null_count = int(data.isna().sum().sum()) 
print(f"Null count: {null_count}")

Null count: 972


In [4]:
missing_values = pd.DataFrame(data.isnull().sum().sort_values(ascending=False)) # type: ignore
missing_values

,0
property_damage,360
police_report_available,343
collision_type,178
authorities_contacted,91
months_as_customer,0
age,0
policy_number,0
policy_deductable,0
policy_csl,0
policy_state,0


In [5]:
target = 'total_claim_amount'
X_train, X_test, X_val, y_train, y_test, y_val = pr.split_data(data, target)# type: ignore

In [7]:
data['policy_bind_date'] = pd.to_datetime(data['policy_bind_date'], yearfirst=True)
data['month'] = data['policy_bind_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['month']/12)
data['month_cos'] = np.cos(2 * np.pi * data['month']/12)

data = data.drop('month', axis=1)

In [8]:

data['insured_zip'] = data['insured_zip'].astype(object)
date_cols = data.select_dtypes(include="datetime").columns# type: ignore
bool_cols = []
for col in data.columns:
    unique_vals = set(data[col].dropna().unique())
    if unique_vals.issubset({True, False}):
        data[col] = data[col].astype('boolean')  # nullable bool
        bool_cols.append(col)

for col in bool_cols:
    print(f"Bool Column --------- {col}")
print("\n----------------------------------------------------\n")

for col in date_cols:
    print(f"Date Column --------- {col}")
print("\n----------------------------------------------------\n")
num_cols = data.loc[:,~data.columns.isin(date_cols)].select_dtypes(include="number").columns# type: ignore
for col in num_cols:
    print(f"Numerical Column --------- {col}")

print("\n----------------------------------------------------\n")
cat_cols = data.select_dtypes(include="object").columns# type: ignore
cat_cols = [x for x in cat_cols if x not in bool_cols]
for col in cat_cols:
    print(f"Categorical Column --------- {col}")

num_cols.drop("policy_number")

data = data.drop('policy_bind_date', axis=1)
# Cyclical interpretation of datetime feature

Bool Column --------- property_damage
Bool Column --------- police_report_available
Bool Column --------- fraud_reported

----------------------------------------------------

Date Column --------- policy_bind_date

----------------------------------------------------

Numerical Column --------- months_as_customer
Numerical Column --------- age
Numerical Column --------- policy_number
Numerical Column --------- policy_deductable
Numerical Column --------- policy_annual_premium
Numerical Column --------- umbrella_limit
Numerical Column --------- capital-gains
Numerical Column --------- capital-loss
Numerical Column --------- incident_hour_of_the_day
Numerical Column --------- number_of_vehicles_involved
Numerical Column --------- bodily_injuries
Numerical Column --------- witnesses
Numerical Column --------- total_claim_amount
Numerical Column --------- injury_claim
Numerical Column --------- property_claim
Numerical Column --------- vehicle_claim
Numerical Column --------- auto_year
Nu

In [9]:
print(X_train)

     months_as_customer  age  policy_number policy_bind_date policy_state  \
541                 239   41         743092       2013-11-11           OH   
440                 108   31         492224       2005-12-09           IN   
482                 116   30         996253       2001-11-29           IN   
422                   8   21         355085       2012-10-09           IN   
778                 161   38         192524       2004-01-02           IL   
..                  ...  ...            ...              ...          ...   
106                 464   61         632627       1990-10-07           OH   
270                 369   55         577810       2013-04-15           OH   
860                 230   42         175960       2004-11-16           IN   
435                 102   28         810189       1999-08-29           OH   
102                 279   41         389238       2001-06-06           IL   

    policy_csl  policy_deductable  policy_annual_premium  umbrella_limit  \

In [10]:
numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
ordinal_columns = ['insured_sex', 'insured_education_level', 'incident_type','incident_severity',]
ordinal_categories = [['MALE', 'FEMALE'],
                      ['High School', 'Masters','JD', 'MD'],
                      ['Single Vehicle Collision', 'Multi-vehicle Collision'],
                      ['Minor Damage', 'Major Damage', 'Total Loss']]
nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [33]:
print(nominal_columns)

['policy_state', 'policy_csl', 'insured_zip', 'insured_occupation', 'insured_hobbies', 'insured_relationship', 'incident_date', 'collision_type', 'authorities_contacted', 'incident_state', 'incident_city', 'incident_location', 'auto_make', 'auto_model']


In [75]:
importlib.reload(pr)
preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)

In [76]:
importlib.reload(pr)
global_mean = y_train.mean()
X_val_processed = pr.apply_preprocessing_pipeline(X_val,
                                                  preprocessor=preprocessor)

X_test_processed = pr.apply_preprocessing_pipeline(X_test,
                                                  preprocessor=preprocessor)

In [78]:
train_data = pd.DataFrame(X_train_processed)
test_data = pd.DataFrame(X_test_processed)
val_data = pd.DataFrame(X_val_processed)
processed_data = pd.concat([
    train_data,
    test_data,
    val_data
], ignore_index=True)   

In [79]:
processed_data.shape

(1000, 38)

In [80]:
print(processed_data.isnull().sum().sum())

0


In [81]:
i = 0
for df in [train_data, test_data, val_data]:
    print(f"Null values in {i}: {df.isnull().sum().sum()}")
    i += 1

Null values in 0: 0
Null values in 1: 0
Null values in 2: 0
